In [1]:
import numpy as np
np.random.seed(1234)

In [2]:
class VBPCA:
    def __init__(self, max_iter=1000):
        self.max_iter = max_iter
        self.a_gamma0 = 1e-6
        self.b_gamma0 = 1e-6
        self.initial_r = 50
        self.prune_thres = 50
        
    def initialize(self, Y):
        self.m, self.n = Y.shape
#         self.r = np.min([self.m, self.n])
        self.r = self.initial_r
        U, S, VT = np.linalg.svd(Y, full_matrices=True)
        S_sqrt = np.sqrt(S)
        self.A = (U * S_sqrt.reshape(1, -1))[:, :self.r]
        self.B = (VT.T * S_sqrt.reshape(1, -1))[:, :self.r]
        scale = np.sqrt(np.mean(Y**2))
        self.Sigma_A = scale * np.eye(self.r)
        self.Sigma_B = scale * np.eye(self.r)
        self.gamma = np.ones(self.r) * scale
        self.beta = 1 / scale**2
        self.Sigma_E = scale * np.ones([self.r, self.r])
        self.alpha = scale * np.ones([self.m, self.n])
        self.X = self.A @ self.B.T
        self.E = Y - self.X
        
    def run_one_iter(self, Y):
        Gamma = np.diag(self.gamma)
        # A
        self.Sigma_A = np.linalg.inv(self.beta * self.B.T @ self.B + self.beta * self.n * self.Sigma_B + Gamma)
        self.A = self.beta * (Y - self.E) @ self.B @ self.Sigma_A
        # B
        self.Sigma_B = np.linalg.inv(self.beta * self.A.T @ self.A + self.beta * self.m * self.Sigma_A + Gamma)
        self.B = self.beta * (Y - self.E).T @ self.A @ self.Sigma_B
        # X
        X_new = self.A @ self.B.T
        self.X_diff = np.mean((X_new - self.X)**2)
        self.X = X_new
        # E
        self.Sigma_E = 1 / (self.alpha + self.beta)
        self.E = self.beta * (Y - self.E) * self.Sigma_E
        # alpha
        self.alpha = 1 / (self.E**2 + self.Sigma_E)
        # gamma
        self.gamma = (self.m + self.n + self.a_gamma0) \
                     / (self.b_gamma0 \
                        + np.diagonal(self.B.T @ self.B) + self.n * np.diagonal(self.Sigma_B) \
                        + np.diagonal(self.A.T @ self.B) + self.m * np.diagonal(self.Sigma_A))
        # beta
        err = np.sum((Y - self.X - self.E)**2) \
              + self.n * np.trace(self.A.T @ self.A @ self.Sigma_B) \
              + self.m * np.trace(self.B.T @ self.B @ self.Sigma_A) \
              + self.m * self.n * np.trace(self.Sigma_A @ self.Sigma_B) \
              + np.sum(self.Sigma_E)
        self.beta = (self.m * self.n) / err
        
        # prune
        max_gamma = np.min(self.gamma) * self.prune_thres
        if np.sum(self.gamma > max_gamma):
            index = (self.gamma <= max_gamma)
            self.A = self.A[:, index]
            self.B = self.B[:, index]
            self.gamma = self.gamma[index]
            self.Sigma_A = self.Sigma_A[index]
            self.Sigma_A = self.Sigma_A[:, index]
            self.Sigma_B = self.Sigma_B[index]
            self.Sigma_B = self.Sigma_B[:, index]
            self.r = len(self.gamma)

    def fit(self, Y):
        self.initialize(Y)
        for t in range(self.max_iter):
            self.run_one_iter(Y)
            if not (t + 1) % 100:
                print('The %d-th iteration finished.' % (t + 1))
                print('Rank=%d, X_difference=%.3fx1e-10' % (self.r, self.X_diff*1e10))
            if self.X_diff < 1e-10:
                print('***Converge at step %d***' % (t + 1))
                break
        self.r_hat = np.linalg.matrix_rank(self.X)
        print('Estimated Rank=%d' % self.r_hat)

In [3]:
r_list = [5, 10, 15, 20, 25, 30, 35, 40]
estimated_r_list = []
x_error_list = []
p = 0.01
for r in r_list:
    A = np.random.normal(0, 1, [300, r])
    B = np.random.normal(0, 1, [300, r])
    X = A @ B.T
    P = np.random.binomial(1, p, [300, 300])
    E_dense = np.random.uniform(-10, 10, [300, 300])
    E = P * E_dense
    N = np.random.normal(0, 0.05, [300, 300])
    Y = X + E + N
    
    model_VBPCA = VBPCA()
    model_VBPCA.fit(Y)
    estimated_r_list.append(model_VBPCA.r_hat)
    x_error_list.append(np.mean(model_VBPCA.X - X)**2)

The 100-th iteration finished.
Rank=5, X_difference=116527.226x1e-10
The 200-th iteration finished.
Rank=5, X_difference=1735.905x1e-10
The 300-th iteration finished.
Rank=5, X_difference=56.023x1e-10
The 400-th iteration finished.
Rank=5, X_difference=3.063x1e-10
***Converge at step 443***
Estimated Rank=5
The 100-th iteration finished.
Rank=10, X_difference=1574008.326x1e-10
The 200-th iteration finished.
Rank=10, X_difference=274827.394x1e-10
The 300-th iteration finished.
Rank=10, X_difference=55486.114x1e-10
The 400-th iteration finished.
Rank=10, X_difference=12283.034x1e-10
The 500-th iteration finished.
Rank=10, X_difference=2875.793x1e-10
The 600-th iteration finished.
Rank=10, X_difference=698.861x1e-10
The 700-th iteration finished.
Rank=10, X_difference=174.371x1e-10
The 800-th iteration finished.
Rank=10, X_difference=44.354x1e-10
The 900-th iteration finished.
Rank=10, X_difference=11.446x1e-10
The 1000-th iteration finished.
Rank=10, X_difference=2.986x1e-10
Estimated Ra

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure()
plt.style.use('ggplot')
plt.plot(r_list, estimated_r_list)
plt.scatter(r_list, estimated_r_list)
plt.xlabel("True Ranks")
plt.ylabel("Estimated Ranks")
plt.xlim((0, 45))
plt.ylim((0, 45))
plt.savefig("VBPCA_rank.pdf", dpi=300, quality=95)

plt.figure()
plt.style.use('ggplot')
plt.plot(r_list, [e * 1e5 for e in x_error_list])
plt.scatter(r_list, [e * 1e5 for e in x_error_list])
plt.xlabel("True Ranks")
plt.ylabel("Error x 1e-5")
plt.xlim((0, 45))
plt.savefig("VBPCA_error.pdf", dpi=300, quality=95)